In [1]:
import requests
from faker import Faker
from faker import Factory
import ujson as json
import pandas as pd
import requests
import certifi
from numpy import random 
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch_dsl import Search, DocType, Date, Integer, Keyword, Text
from datetime import datetime
from elasticsearch_dsl.connections import connections
import pandas as pd
import requests
from random import randint
from datetime import datetime, timedelta
import math
import numpy as np
from pandas.io.json import json_normalize
      
      



In [2]:
ES_HOST = 'http://team.ankit.one'
INDEX = "eda_small"
DOC_TYPE="user"
es = Elasticsearch(ES_HOST)

In [3]:
def get_m_score(var):
    if (var > 50000):
       score = 9
    elif (var > 25000):
       score = 8
    elif (var > 25000):
       score = 7
    elif (var > 10000):
       score = 6
    elif (var > 5000):
       score = 5
    elif (var > 2500):
       score = 4
    elif (var > 1250):
       score = 3
    elif (var > 750):
       score = 2
    elif (var > 250):
       score = 1
    else:
       score = 0
    return score

In [4]:
def get_f_score(var):
    if (var > 50):
       score = 9
    elif (var > 40):
       score = 8
    elif (var > 30):
       score = 7
    elif (var > 20):
       score = 6
    elif (var > 10):
       score = 5
    elif (var > 5):
       score = 4
    elif (var > 3):
       score = 3
    elif (var > 2):
       score = 2
    elif (var > 1):
       score = 1
    else:
       score = 0
    return score

In [5]:
def get_r_score(var):
    if (var < 10):
       score = 9
    elif (var < 15 ):
       score = 8
    elif (var < 30):
       score = 7
    elif (var < 45):
       score = 6
    elif (var < 60):
       score = 5
    elif (var < 90):
       score = 4
    elif (var < 120):
       score = 3
    elif (var < 240):
       score = 2
    elif (var < 365):
       score = 1
    else:
       score = 0
    return score

In [6]:
def get_lifecycle(orders, days_last_purchase, debug = False):
    if (orders == 0):
        lc = "Contact"
        if (debug == True):
            print("Orders is {}, Days Last Purchase is {}, and Lifecycle is {}".format(orders, days_last_purchase,lc))
    elif (orders < 2    and days_last_purchase <= 21):
        lc = "One time (Active)"
        if (debug == True):
            print("Orders is {}, Days Last Purchase is {}, and Lifecycle is {}".format(orders, days_last_purchase,lc))
    elif (orders < 2   and days_last_purchase <= 60):
        lc = "One time (Sleeping)"
        if (debug == True):
            print("Orders is {}, Days Last Purchase is {}, and Lifecycle is {}".format(orders, days_last_purchase,lc))
    elif (orders < 2    and days_last_purchase > 60):
        lc = "One time (Churned)"
        if (debug == True):
            print("Orders is {}, Days Last Purchase is {}, and Lifecycle is {}".format(orders, days_last_purchase,lc))
    elif (orders <= 4 and days_last_purchase <= 21):
        lc = "Repeat (Active)"
        if (debug == True):
            print("Orders is {}, Days Last Purchase is {}, and Lifecycle is {}".format(orders, days_last_purchase,lc))
    elif (orders <= 4 and days_last_purchase <= 60):
        lc = "Repeat (Sleeping)"
        if (debug == True):
            print("Orders is {}, Days Last Purchase is {}, and Lifecycle is {}".format(orders, days_last_purchase,lc))
    elif (orders <= 4 and days_last_purchase > 60):
        lc = "Repeat (Churned)"
        if (debug == True):
            print("Orders is {}, Days Last Purchase is {}, and Lifecycle is {}".format(orders, days_last_purchase,lc))
    elif (days_last_purchase <= 21):
        lc = "Loyal (Active)"
        if (debug == True):
            print("Orders is {}, Days Last Purchase is {}, and Lifecycle is {}".format(orders, days_last_purchase,lc))
    elif (orders > 4 and days_last_purchase <= 60):
        lc = "Loyal (Sleeping)"
        if (debug == True):
            print("Orders is {}, Days Last Purchase is {}, and Lifecycle is {}".format(orders, days_last_purchase,lc))
    elif (orders > 4 and days_last_purchase > 60):
        lc = "Loyal (Churned)"
        if (debug == True):
            print("Orders is {}, Days Last Purchase is {}, and Lifecycle is {}".format(orders, days_last_purchase,lc))
    else:
        lc ="Others"
    return lc
        


In [8]:
def get_city():
    cities = ["Mumbai","Delhi","Bangalore","Hyderabad","Ahmedabad","Chennai","Kolkata","Surat","Pune","Jaipur",
    "Lucknow","Kanpur","Nagpur","Indore","Thane","Bhopal","Visakhapatnam","Noida","Patna","Vadodara"]
    p=[0.2,0.15,0.125,0.1,0.125,0.05, 0.05, 0.025,0.025,0.025,0.025,0.02,0.01, 0.01,0.01,0.01,0.01, 0.01,0.01,0.01]
    return np.random.choice( cities, 1, p)[0]


In [9]:
def get_segment(city, revenue, discount_percentage, aov):
    segments = []
    if (city == 'Delhi' or city =='Noida'):
        segment = 'NCR'
        segments.append(segment)
        
    if (revenue > 75000):
        segment = 'VIP Customers'
        segments.append(segment)
        
    if (discount_percentage > 25):
        segment = 'High Discount Seeker'
        segments.append(segment)
        
    if (aov > 3000):
        segment = 'High Cart Value'
        segments.append(segment)
        
   
    return segments

In [15]:
fake = Factory.create('en')
j = 0

#An empty list
actions = []

while (j <= 260):
    revenue = random.randint(3050, 10000)
    #revenue = 0
    num_of_orders = math.ceil(revenue / random.randint(300, 5000))
    m_score = get_m_score(revenue)
    f_score = get_f_score(num_of_orders)
    if (num_of_orders == 0):
        aov = 0
    else:
        aov = revenue / num_of_orders
    days_since_last_purchase = random.randint(1, 100)
    r_score = get_r_score(days_since_last_purchase)
    rfm_score = r_score + f_score + m_score
    discount_percentage = random.randint(5, 30)
    total_discount_revenue = revenue * discount_percentage /100
    lifecycle = get_lifecycle(num_of_orders, days_since_last_purchase, False )
    name = fake.name()
    city = get_city()
    segments = []
    segments = get_segment(city, revenue, discount_percentage, aov)
    if (random.randint(0, 100) > 93):
        email_unsubscribe = True
    else:
        email_unsubscribe = False
                
    action = {
        "_index": INDEX,
        '_op_type': 'create',
        "_type": DOC_TYPE,
        "_id": fake.uuid4(),
        "_source": {
            "name": name,
            "email_unsubscribe": email_unsubscribe,
            "revenue" : revenue,
            "days_since_last_purchase" : days_since_last_purchase,
            "r_score": r_score,
            "f_score": f_score,
            "m_score": m_score,
            "rfm_score": rfm_score,
            "num_of_orders": num_of_orders,
            "discount_percentage": discount_percentage,
            "total_discount_revenue": total_discount_revenue,
            "lifecycle":lifecycle,
            "segments":segments,
            "Avg Order Value": aov,
            "City":city
            }
        }
    
    actions.append(action)
    j += 1
    

    while (len(actions) > 500):
        helpers.bulk(es, actions)
        del actions[0:len(actions)]
        print("500 more records inserted...")
        # Empty the actions list after 10,000 times
        break

# Some data is remaining since the loop has gone when len(actions) is less than 10,000
if (len(actions) > 0):
    helpers.bulk(es, actions)
    del actions[0:len(actions)]
print("All records inserted")
 

All records inserted


In [ ]:
# save match all query as python variable
myquery={"from" : 0, "size" : 2000,
         "query": 
         {"match_all": {}}
        }

# execute the query using body parameter and return total number of records
# select count(*) from table
res = es.search(index=INDEX, body=myquery) 
df = json_normalize(res['hits']['hits'])
df_index = df.set_index(['_id'])
df_index.drop(['_index', '_score', '_type'],inplace=True,axis=1,errors='ignore')
df_index['_source.lifecycle'] = df_index['_source.lifecycle'].astype(np.int64)
data = df_index['_source.lifecycle']
plt.hist(data, bins=np.arange(data.min(), data.max()+1), normed=False)
plt.show()


#plt.dist(roll_options,stats.randint.pmf(roll_options,low_value,high_value))

sns.distplot(df_index['_source.discount_percentage'],bins=10, kde=True,  rug=True)
plt.show()


- Should we have seperate index for each projects?
- If a client becomes inactive? It's data is still there. 